In [40]:
import pandas as pd
import geopandas as gpd
import altair as alt
import numpy as np
import os
import chromatose as ct

import geopandas as gpd
import json

from IPython.display import SVG
alt.renderers.set_embed_options(actions=False, renderer='svg')

RendererRegistry.enable('default')

In [31]:
df_compare_ij = pd.read_csv('processed_data/DF_COMPARE_IJ.csv')
df_compare_i = pd.read_csv('processed_data/DF_COMPARE_I.csv')
df_compare_j = pd.read_csv('processed_data/DF_COMPARE_J.csv')

In [ ]:
# used for centroid calculation, simplify used for plot
state_tracts = gpd.read_file(
    'processed_data/california_tracts_2020.geojson')

state_tracts_simplify = gpd.read_file(
    'processed_data/california_tracts_2020_simplify.geojson')
state_tracts_simplify = state_tracts_simplify.to_crs(epsg=4326)

state_counties = gpd.read_file('processed_data/california_counties_2020.geojson')
state_counties = state_counties.to_crs(epsg=4326)

pali_fire = gpd.read_file('processed_data/palisades_fire_perimeter.geojson')
eaton_fire = gpd.read_file('processed_data/eaton_fire_perimeter.geojson')
pali_fire = pali_fire.set_crs(epsg=3857, allow_override=True).to_crs(epsg=4326)
eaton_fire = eaton_fire.set_crs(epsg=3857, allow_override=True).to_crs(epsg=4326)

In [69]:
# df_compare_ij = df_compare_ij.merge(df_compare_i[['county_i_name', 'L_i_data']], how='left', on='county_i_name')
# # proportion of change in L_ij over 
# df_compare_ij['diff_L_ij_prop'] = df_compare_ij['diff_L_ij'] / df_compare_ij['L_i_data']

# palette = ct.palpolate(ct.nacre[:-1])[::-1]
max_val = df_compare_ij['diff_L_ij'].abs().max() 
heatmap_diff_counts = (alt.Chart(df_compare_ij).mark_rect(stroke='#cdcdcd', strokeWidth=1).encode(
        x=alt.X("county_i_name:N", title="Resident county", axis=alt.Axis(labelAngle=45)),
        y=alt.Y("county_j_name:N", title="Work county"),
        color=alt.Color(
            "diff_L_ij:Q",
            title="Δ in # of workers",
            scale=alt.Scale(
                scheme='redblue',
                # type='symlog', 
                domain=[-1e6, 1e6], clamp=True)  # important for skewed flows
        ), tooltip=["county_i_name:N","county_j_name:N","diff_L_ij:Q"]
    ).properties(width=300, height=300, title="Δ in commuting flows: cf - eq.")
)

heatmap_diff_counts

alt.Chart(...)

The heatmap above shows the net outflow from workerse in LA, who after the shock go on to work in Ventura and Orange County.
We see a strong diagonal bc even small changes in $\pi_{ij}$ multiplied by large resident populations produce a big $\Delta L_{ii}$

In [73]:
d_county_FID = {
    'Los Angeles':1993, 
    'Orange':362, 
    'Riverside':2943, 
    'San Bernardino':817,
    'San Diego':1896, 
    'Ventura':963
}

df_compare_i['FID'] = df_compare_i['county_i_name'].map(d_county_FID)
df_compare_j['FID'] = df_compare_j['county_j_name'].map(d_county_FID)

df_compare_i = df_compare_i.merge(state_counties, how='left', on='FID')
df_compare_j = df_compare_j.merge(state_counties, how='left', on='FID')

In [94]:
# geojson = json.loads(df_compare_i.to_json())
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": geom.__geo_interface__,
            "properties": row.drop("geometry").to_dict(),
        }
        for _, row in df_compare_i.iterrows()
        for geom in [row.geometry]
    ]
}

In [123]:
map_p_i_eq = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#cdcdcd', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.p_i_eq:Q", title="p_i (eq)",
            scale=alt.Scale(scheme='reds'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='p_i EQ')
)

map_p_i_cf = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.p_i_cf:Q", title="p_i (cf)",
            scale=alt.Scale(scheme='reds'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='p_i CF')
)

map_p_i_diff = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.diff_p_i:Q", title="Δ p_i", 
            scale=alt.Scale(scheme='redblue', domainMid=0))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='Δ p_i, CF - EQ')
)

In [124]:
alt.hconcat(map_p_i_eq, map_p_i_cf)

alt.HConcatChart(...)

In [125]:
map_p_i_diff

alt.Chart(...)

In [126]:
# plot L_j_eq, L_j_cf , change in L_j

In [130]:
# geojson = json.loads(df_compare_i.to_json())
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": geom.__geo_interface__,
            "properties": row.drop("geometry").to_dict(),
        }
        for _, row in df_compare_j.iterrows()
        for geom in [row.geometry]
    ]
}

map_L_j_eq = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#cdcdcd', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.L_j_eq:Q", title="L_j (eq)",
            scale=alt.Scale(scheme='reds'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='L_j EQ')
)

map_L_j_cf = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.L_j_cf:Q", title="L_j (cf)",
            scale=alt.Scale(scheme='reds'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='L_j CF')
)

map_L_j_diff = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.diff_L_j:Q", title="Δ L_j", 
            scale=alt.Scale(scheme='redblue', domainMid=0))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='Δ p_i, CF - EQ')
)

In [132]:
alt.hconcat(map_L_j_eq, map_L_j_cf)

alt.HConcatChart(...)

In [133]:
map_L_j_diff

alt.Chart(...)